In [15]:
import numpy as np
import pandas as pd
from keras import Model
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization, Input

## Data

In [16]:
df_train = pd.read_csv("./clean_train.csv")
df_train.head()

,Tenure,Matric,Degree,Diploma,Female,Sa_citizen,Birthyear,Birthmonth,Target,Year_survey,...,Home_lang_70_79,Home_lang_80_100,Home_lang_nan,Science_30_39,Science_40_49,Science_50_59,Science_60_69,Science_70_79,Science_80_100,Science_nan
0,0.0,1.0,0.0,0.0,1,1,2000,5,0,2022,...,False,False,True,False,False,False,False,False,False,False
1,427.0,1.0,0.0,0.0,1,1,1989,4,1,2023,...,False,False,True,True,False,False,False,False,False,False
2,0.0,1.0,0.0,0.0,0,1,1996,7,1,2022,...,False,False,True,True,False,False,False,False,False,False
3,810.0,0.0,0.0,0.0,0,1,2000,1,0,2022,...,False,False,True,False,False,False,False,False,False,True
4,0.0,0.0,0.0,0.0,1,1,1998,12,0,2023,...,False,False,True,False,False,False,False,False,False,True


In [17]:
df_train.shape

(4020, 74)

In [18]:
train = df_train.drop('Target', axis=1)
targets = df_train['Target']

train.shape, targets.shape

((4020, 73), (4020,))

## Test Data

In [19]:
# unclean data
test_unclean_df = pd.read_csv("./Test.csv")
test_unclean_df.head()

,Person_id,Survey_date,Round,Status,Tenure,Geography,Province,Matric,Degree,Diploma,Schoolquintile,Math,Mathlit,Additional_lang,Home_lang,Science,Female,Sa_citizen,Birthyear,Birthmonth
0,Id_r90136smvl,2022-08-03,3,other,NaN,Urban,KwaZulu-Natal,1.0,0.0,0.0,2.0,0 - 29 %,NaN,50 - 59 %,NaN,40 - 49 %,0,1,2002,12
1,Id_wawdqhmu6s,2023-03-16,4,unemployed,979.0,Urban,Western Cape,1.0,0.0,0.0,NaN,NaN,NaN,40 - 49 %,NaN,NaN,1,1,1989,12
2,Id_ap2czff2bu,2023-03-14,4,unemployed,339.0,Urban,KwaZulu-Natal,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1,1989,12
3,Id_uhgink7iha,2023-02-16,4,studying,NaN,Urban,Gauteng,1.0,0.0,0.0,1.0,NaN,80 - 100 %,60 - 69 %,NaN,NaN,0,1,2002,11
4,Id_5j6bzk3k81,2023-03-23,4,unemployed,613.0,Urban,Gauteng,0.0,0.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,1,1,1993,10


In [20]:
# clean data
test_data = pd.read_csv("./clean_test.csv")
test_data.head()

,Tenure,Matric,Degree,Diploma,Female,Sa_citizen,Birthyear,Birthmonth,Year_survey,Age_survey,...,Home_lang_70_79,Home_lang_80_100,Home_lang_nan,Science_30_39,Science_40_49,Science_50_59,Science_60_69,Science_70_79,Science_80_100,Science_nan
0,0.0,1.0,0.0,0.0,0,1,2002,12,2022,20,...,False,False,True,False,True,False,False,False,False,False
1,979.0,1.0,0.0,0.0,1,1,1989,12,2023,34,...,False,False,True,False,False,False,False,False,False,True
2,339.0,0.0,0.0,0.0,1,1,1989,12,2023,34,...,False,False,True,False,False,False,False,False,False,True
3,0.0,1.0,0.0,0.0,0,1,2002,11,2023,21,...,False,False,True,False,False,False,False,False,False,True
4,613.0,0.0,0.0,0.0,1,1,1993,10,2023,30,...,False,False,True,False,False,False,False,False,False,True


## Models

In [21]:
predictions = []

#### Naive Bayes Gaussian

In [22]:
# # NB
# grid_search_params = {"var_smoothing": [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}

# cls_gnb = GaussianNB()

# grid_search = GridSearchCV(
#     estimator=cls_gnb, param_grid=grid_search_params, cv=10, scoring="accuracy"
# )

# grid_search.fit(train, targets)

# print(f"{grid_search.best_params_}\n\n")
# print(f"{grid_search.best_estimator_}")

In [23]:
# # perform cross val on data
# gnb = GaussianNB(var_smoothing=1e-07)

# cv = StratifiedKFold(random_state=42, n_splits=10, shuffle=True)

# scores = cross_val_score(gnb, train, targets, cv=cv, scoring='accuracy')

# print(f"mean score: {scores.mean()}")

In [24]:
# gnb.fit(train, targets)
# predictions = gnb.predict(test_data)
# predictions

#### SVM - SVC

In [25]:
# # svc takes times
# # "C": [0.08, 0.05, 0.03, 0.01, 0.1, 1, 3, 5, 7, 10],
# grid_search_params = {
#     "C": [0.01, 0.05, 0.1, 1, 5, 10],
#     "kernel": ["linear", "rbf", "poly", "sigmoid"],
#     "degree": [2, 3, 4, 5],
#     "gamma": ["scale", "auto"] + [0.001, 0.01, 0.1, 1],
# }

# cls_svc = SVC()

# grid_search = GridSearchCV(
#     estimator=cls_svc, param_grid=grid_search_params, cv=5, scoring="accuracy"
# )

# grid_search.fit(train, targets)

# print(f"{grid_search.best_params_}\n\n")
# print(f"{grid_search.best_estimator_}")

#### Deep Learning

In [26]:
def reshape_df(data):
    # Create a sample 2D array
    original_array = np.array(data).astype(np.float32)  # Replace this with your actual data

    # Define the number of time steps (samples)
    num_samples = original_array.shape[0]

    # Define the number of features (variables)
    num_features = original_array.shape[1]

    # Define the desired number of time steps for the 3D array
    desired_num_time_steps = 10  # Change this according to your needs

    # Calculate the number of 3D arrays you can create
    num_3d_arrays = num_samples // desired_num_time_steps

    # Initialize an empty list to store the 3D arrays
    resulting_3d_arrays = []

    # Reshape the data into 3D arrays
    for i in range(num_3d_arrays):
        start_idx = i * desired_num_time_steps
        end_idx = start_idx + desired_num_time_steps
        # Extract a chunk of data for each 3D array
        chunk = original_array[start_idx:end_idx, :]
        # Add an extra dimension to represent time steps (samples)
        chunk = chunk.reshape((desired_num_time_steps, num_features))
        # Append the reshaped chunk to the list
        resulting_3d_arrays.append(chunk)

    # Convert the list of 3D arrays into a single 3D NumPy array
    final_3d_array = np.array(resulting_3d_arrays)

    # Print the shape of the final 3D array
    return np.array( final_3d_array)

dl_data = reshape_df(train)
dl_data.shape

(402, 10, 73)

In [29]:
dl_labels = np.array(targets)
dl_labels.shape

(4020,)

In [27]:
input_shape = (None,73)

input_layer = Input(shape=input_shape)

x = BatchNormalization()(input_layer)

units = [128, 64, 32]

for unit in units:
    x = Bidirectional(LSTM(unit, return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

output_layer = Dense(1, activation="sigmoid")(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# model.summary()

In [ ]:
model.fit(dl_data, dl_labels , batch_size=16, epochs=5, validation_split=0.1)

#### Multinomial NB

In [ ]:
# # NB
# grid_search_params = {
#     "alpha": [0.1, 0.5, 1.0, 2.0],
#     "fit_prior": [True, False],
# }

# cls_mnb = MultinomialNB()

# grid_search = GridSearchCV(
#     estimator=cls_mnb, param_grid=grid_search_params, cv=30, scoring="accuracy"
# )

# grid_search.fit(train, targets)

# print(f"Best Params: {grid_search.best_params_}")
# print(f"Best Estimator: {grid_search.best_estimator_}")
# print(f"Best Score: {grid_search.best_score_}")

In [ ]:
# cls_mnb.fit(train, targets)

In [ ]:
# predictions = cls_mnb.predict(test_data)
# predictions

#### Random Forest Classifier

In [ ]:
# grid_search_params = {
#     'n_estimators': [50, 100, 200],  # Number of trees in the forest
#     'max_depth': [None, 10, 20, 30],  # Maximum depth of each tree
#     'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
#     'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider for the best split
# }

# cls_rf = RandomForestClassifier(random_state=42)

# grid_search = GridSearchCV(
#     estimator=cls_rf, param_grid=grid_search_params, cv=10, scoring="accuracy"
# )

# grid_search.fit(train, targets)

# # print(f"{grid_search.best_params_}\n\n")
# print(f"{grid_search.best_estimator_}")

In [ ]:
# rf = RandomForestClassifier(
#     random_state=42, max_depth=30, min_samples_leaf=2, min_samples_split=5
# )

# cv = StratifiedKFold(random_state=42, n_splits=10, shuffle=True)

# scores = cross_val_score(rf, train, targets, cv=cv, scoring='accuracy')

# print(f"mean score: {scores.mean()}")

mean score: 0.8156716417910447


In [ ]:
# rf.fit(train, targets)
# predictions = rf.predict(test_data)
# predictions

array([1, 0, 0, ..., 0, 0, 0])

## Save to file

In [ ]:

# df_submission = pd.DataFrame({"ID": test_unclean_df["Person_id"], "Target": predictions.astype(int)})
# print(df_submission.head())

              ID  Target
0  Id_r90136smvl       1
1  Id_wawdqhmu6s       0
2  Id_ap2czff2bu       0
3  Id_uhgink7iha       0
4  Id_5j6bzk3k81       0


In [ ]:
# df_submission.to_csv("submission.csv", index=False)